## TOIs

In [1]:
from pathlib import Path
import pandas as pd
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive

def get_tois(
    clobber=False,
    outdir="../data",
    verbose=True,
    remove_FP=True,
):
    """Download TOI list from TESS Alert/TOI Release"""
    dl_link = "https://exofop.ipac.caltech.edu/tess/download_toi.php?sort=toi&output=csv"
    outdir = Path(outdir)
    outdir.mkdir(parents=True, exist_ok=True)
    fp = outdir.joinpath("TOIs.csv")

    if not fp.exists() or clobber:
        msg = f"Downloading TOIs from: {dl_link}"
        if verbose: print(msg)
        d = pd.read_csv(dl_link)
        if remove_FP:
            # remove False Positives
            d = d[d["TFOPWG Disposition"] != "FP"]
            d.to_csv(fp, index=False)
            msg = "TOIs with TFPWG disposition==FP are removed."
            if verbose: print("Saved: ", msg)
    else:
        d = pd.read_csv(fp)
        if verbose: print("Loaded: ", fp)
    
    return d.sort_values("TOI")

def get_nexsci_data(table_name="ps", clobber=False):
    """
    ps: self-consistent set of parameters
    pscomppars: a more complete, though not necessarily self-consistent set of parameters
    """
    url = "https://exoplanetarchive.ipac.caltech.edu/docs/API_PS_columns.html"
    print("Column definitions: ", url)
    fp = f"../data/nexsci_{table_name}.csv"
    if clobber:
        nexsci_tab = NasaExoplanetArchive.query_criteria(table=table_name, where="discoverymethod like 'Transit'")
        df_nexsci = nexsci_tab.to_pandas()
        df_nexsci.to_csv(fp, index=False)
        print("Saved: ", fp)
    else:
        df_nexsci = pd.read_csv(fp)
        print("Loaded: ", fp)
    return df_nexsci

In [2]:
df = get_tois()
df.tail()

Loaded:  ../data/TOIs.csv


,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,SG5,...,Stellar Radius (R_Sun) err,Stellar Metallicity,Stellar Metallicity err,Stellar Mass (M_Sun),Stellar Mass (M_Sun) err,Sectors,Date TOI Alerted (UTC),Date TOI Updated (UTC),Date Modified,Comments
5299,428867906,6060.01,NaN,3,4,3,3,1,4,4,...,0.21,0.086,0.070,1.678,0.294829,"19,59",2023-02-08,2023-02-08,2023-02-10 12:02:44,low SNR; two stars in pixel
5300,239823606,6061.01,NaN,2,4,2,2,1,4,4,...,0.05,NaN,NaN,1.000,0.127632,"19,43,44,45,59",2023-02-08,2023-02-08,2023-02-10 12:02:44,two stars in pixel; low SNR
5301,122629761,6062.01,NaN,2,4,2,2,1,4,4,...,0.05,NaN,NaN,1.084,0.146070,"19,59",2023-02-08,2023-02-08,2023-02-10 12:02:43,variable host; low SNR
5302,67031480,6063.01,NaN,3,4,3,3,1,4,4,...,NaN,NaN,NaN,NaN,NaN,"19,59",2023-02-08,2023-02-08,2023-02-10 12:02:44,likely EB; a/Rs~2; host is member of M38 cluster
5303,428767018,6064.01,NaN,3,4,3,3,1,4,4,...,0.14,0.133,0.012,1.464,0.245446,"19,59",2023-02-08,2023-02-08,2023-02-10 12:02:43,NaN


In [3]:
tics = df['TIC ID'].unique()
len(tics)

5096

In [4]:
len(df['TOI'].unique())

5304

In [5]:
nsys = df['TIC ID'].value_counts()
nsys

367630162    5
425997655    5
150151262    4
260647166    4
251848941    4
            ..
250803548    1
455030519    1
166184428    1
97486178     1
305790119    1
Name: TIC ID, Length: 5096, dtype: int64

In [6]:
tics_multis = list(nsys[nsys>1].index)
print(len(tics_multis))

163


## TOI 1136 (Dai+2023)
First order MMR: 3:2, 2:1, 3:2, 7:5, and 3:2

In [7]:
col = 'Period (days)'
d = df[df['TIC ID']==142276270].sort_values(by=col)
d[col]

830     6.258792
829    12.518728
832    18.804756
831    26.317954
Name: Period (days), dtype: float64

In [8]:
# add two outer planets
d = pd.DataFrame({col: [4.17278,
                6.25725,
                12.51937,
                18.7992,
                26.3162,
                39.5387]
    })
d

,Period (days)
0,4.17278
1,6.25725
2,12.51937
3,18.79920
4,26.31620
5,39.53870


https://arxiv.org/abs/1208.3312

$$
\frac{p'}{p} \sim \frac{j}{j-1}
$$

proximity to resonance:
$$
\Delta = \frac{p'}{p} \frac{j-1}{j} - 1
$$

In [9]:
import numpy as np
from itertools import combinations

def get_planet_pairs(Nplanets, order=1):
    cs = list(combinations(range(Nplanets), 2))
    cs = [(i,j) for i,j in cs if abs(i-j)==order]
    return cs

def get_orbit_pairs(N=10):
    cs = list(combinations(np.arange(1, N), 2))
    return cs

def get_resonant_pairs(d, order=1, tol=0.01):
    Nplanets = len(d)
    ordered_pairs = get_planet_pairs(Nplanets, order=order)
    deltas = []
    for i_out,i_in in ordered_pairs:
        Pout = d.iloc[i_out].squeeze()
        Pin = d.iloc[i_in].squeeze()
        for i,j in get_orbit_pairs(10):            
            delta = abs((Pout/Pin)*(j/i) - 1)
            if delta<=tol:
                text = f"{j}:{i} | P=({Pout:.2f},{Pin:.2f}) n=({i_in+1},{i_out+1}) (delta={delta*100:.2f}%)"
                deltas.append(text)
                break
    return deltas

In [10]:
get_planet_pairs(6, order=1)

[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)]

In [11]:
resonances_1st_order = get_resonant_pairs(d, order=1, tol=0.01)
resonances_1st_order

['3:2 | P=(4.17,6.26) n=(2,1) (delta=0.03%)',
 '2:1 | P=(6.26,12.52) n=(3,2) (delta=0.04%)',
 '3:2 | P=(12.52,18.80) n=(4,3) (delta=0.11%)',
 '7:5 | P=(18.80,26.32) n=(5,4) (delta=0.01%)',
 '3:2 | P=(26.32,39.54) n=(6,5) (delta=0.16%)']

In [12]:
get_planet_pairs(6, order=2)

[(0, 2), (1, 3), (2, 4), (3, 5)]

In [13]:
resonances_2nd_order = get_resonant_pairs(d, order=2, tol=0.01)
resonances_2nd_order

['3:1 | P=(4.17,12.52) n=(3,1) (delta=0.01%)',
 '3:1 | P=(6.26,18.80) n=(4,2) (delta=0.15%)']

## All TOIs

In [21]:
tois_ttv = []
tois_ttv_multi = []
tois_ttv_with_second_order_too = []
for tic in tics_multis:
    dd = df[df['TIC ID']==tic].squeeze()
    d = dd["Period (days)"]
    toi = str(dd.iloc[0].TOI).split('.')[0] 
    resonances_1st_order = get_resonant_pairs(d, order=1, tol=0.01);
    resonances_2nd_order = get_resonant_pairs(d, order=2, tol=0.01);
    if len(resonances_1st_order)>=1:
        print(f"TOI {toi} | TIC {tic}:")
        print(f"First Order: {[i for i in resonances_1st_order]}\n")
        tois_ttv.append(toi)
        if len(resonances_1st_order)>=2:
            tois_ttv_multi.append(toi)
    if len(resonances_2nd_order)>=1:
        print(f"Second Order: {[i for i in resonances_2nd_order]}\n")
        tois_ttv_with_second_order_too.append(toi)

TOI 2104 | TIC 367630162:
First Order: ['9:5 | P=(3.27,5.90) n=(2,1) (delta=0.24%)']

TOI 174 | TIC 425997655:
First Order: ['2:1 | P=(3.98,7.91) n=(5,4) (delta=0.58%)']

TOI 282 | TIC 29781292:
First Order: ['8:3 | P=(31.32,84.26) n=(3,2) (delta=0.87%)']

Second Order: ['3:2 | P=(56.01,84.26) n=(3,1) (delta=0.30%)']

TOI 700 | TIC 150428135:
First Order: ['7:3 | P=(16.05,37.42) n=(2,1) (delta=0.08%)']

Second Order: ['3:1 | P=(6.26,18.80) n=(4,2) (delta=0.15%)']

Second Order: ['2:1 | P=(6.23,12.57) n=(3,1) (delta=0.91%)']

TOI 1339 | TIC 269701147:
First Order: ['4:3 | P=(28.58,38.35) n=(3,2) (delta=0.64%)']

TOI 2269 | TIC 441798995:
First Order: ['2:1 | P=(1.42,2.84) n=(2,1) (delta=0.00%)']

Second Order: ['3:1 | P=(40.75,123.06) n=(3,1) (delta=0.66%)']

TOI 699 | TIC 149302744:
First Order: ['9:4 | P=(14.80,33.63) n=(2,1) (delta=0.99%)']

TOI 270 | TIC 259377017:
First Order: ['2:1 | P=(5.66,11.38) n=(2,1) (delta=0.51%)']

Second Order: ['9:4 | P=(6.39,14.33) n=(3,1) (delta=0.28%)

/home/jp/miniconda3/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in double_scalars
/home/jp/miniconda3/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in double_scalars
/home/jp/miniconda3/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars
/home/jp/miniconda3/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in double_scalars
/home/jp/miniconda3/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars
/home/jp/miniconda3/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in double_scalars
/home/jp/miniconda3/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in double_sc

In [17]:
tois_ttv = sorted(map(int, tois_ttv))
tois_ttv

[174,
 175,
 270,
 282,
 663,
 696,
 699,
 700,
 714,
 1097,
 1208,
 1339,
 1692,
 1749,
 2095,
 2104,
 2269,
 2540,
 4323,
 5520,
 6054]

In [18]:
np.savetxt('../tql/tois.txt', tois_ttv, fmt="%d")

In [23]:
tois_ttv_multi

['663']

In [24]:
sorted(map(int, tois_ttv_with_second_order_too))

[282, 880, 1136, 1670, 1730]

## Transit timing database
* https://ui.adsabs.harvard.edu/abs/2022ApJS..259...62I/abstract

In [103]:
#https://github.com/transit-timing/tt/blob/master/3_database/table4.csv
url = "https://raw.githubusercontent.com/transit-timing/tt/master/3_database/table4.csv"
df2 = pd.read_csv(url)
df2

,System,Orbit number,T_mid,Uncertainty (days),Time System,#,Reference
0,CoRoT-01,-1412,2.454138e+06,0.000470,BJD,1,2009A&A...506..369B
1,CoRoT-01,-1411,2.454140e+06,0.000590,BJD,1,2009A&A...506..369B
2,CoRoT-01,-1410,2.454141e+06,0.000620,BJD,1,2009A&A...506..369B
3,CoRoT-01,-1409,2.454143e+06,0.000390,BJD,1,2009A&A...506..369B
4,CoRoT-01,-1408,2.454144e+06,0.001630,BJD,1,2009A&A...506..369B
...,...,...,...,...,...,...,...
8662,XO-7,214,2.459407e+06,0.000510,BJD_TDB,1,This work
8663,XO-7,215,2.459410e+06,0.000467,BJD_TDB,1,This work
8664,XO-7,216,2.459413e+06,0.000508,BJD_TDB,1,This work
8665,XO-7,217,2.459415e+06,0.000517,BJD_TDB,1,This work
